In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchFrameException

In [2]:
import pandas as pd
import numpy as np
import re
import time

In [7]:
def cafe_crawling2():
    description_state = False
    try:
        description_a_link = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt")
        description_a_link.click()
        description_state = True
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    

    image_div = soup.find('div', {'id': 'ibu_1'})
    if image_div:
        style = image_div.get('style')
        background_image_url = re.search(r'url\((.*?)\)', style).group(1)
    else:
        background_image_url = ""
    
    # reviews_div = soup.find('div', class_='dAsGb')
    # starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    # if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
    #     return None
    # visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    # blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    try:
        # 별점 요소 찾기
        star_rating_element = driver.find_element(By.CSS_SELECTOR, "div.dAsGb > span.PXMot.LXIwF")
        starring = float(star_rating_element.text.strip()[2:])
    except NoSuchElementException:
        # 별점 요소가 없는 경우
        starring = 0

    try:
        # 방문자 리뷰 요소 찾기
        visitor_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/visitor')]")
        visitor_reviews = int(visitor_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 방문자 리뷰 요소가 없는 경우
        visitor_reviews = 0
        
    try:
        # 블로그 리뷰 요소 찾기
        blog_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/ugc')]")
        blog_reviews = int(blog_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 블로그 리뷰 요소가 없는 경우
        blog_reviews = 0
    
    if visitor_reviews < 10:
        return None
    #PIbes > O8qbU > vV_z_ > zPfVt
    # description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else ""
    if description_state:
        description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt").text
    else:
        try:
            description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > div.xHaT3 >  span.zPfVt").text
        except:
            description = ""
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    keyword_tab_link.click()
    
    keyword_state = False
    idx = 0
    while not keyword_state and idx < 5:
        try:
            keyword_list_link = WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
            keyword_list_link.click()
            keyword_state = True
        except:
            for _ in range(5):
                keyword_tab_link.click()
                time.sleep(0.5)
        idx+=1
    
#     for _ in range(3):
#         try:
#             keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
#             keyword_list_link.click()
#             keyword_state = True
#         except:
#             keyword_tab_link.click()
    
    if not keyword_state:
        return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", "", "", background_image_url]

    while True:
        try:
            keyword_more_link = driver.find_element(By.CLASS_NAME, 'Tvx37')
            keyword_more_link.click()
        except Exception as ex:
            break

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(driver.find_element(By.CSS_SELECTOR, "div._Wmab > em").text.replace(",", ""))

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number, background_image_url]

In [4]:
cafe_search_list= [
                   "동성로", "대구 중앙로", "교동", "경북대병원역",
                    "동대구역", "대구 평화시장", "안심역",
                    "대구 광장코아", "만평역", "대구 서구청",
                    "명덕역", "대구 앞산",
                    "경북대 북문", "대구 복현오거리", "동천동 카페", "칠곡네거리",
                    "범어네거리", "수성유원지", "시지", "지산",
                    "두류공원", "용산", "월배", "계명대"
                    ]

In [9]:
for cafe_name in ["계명대", "동대구역", "경북대병원역"]:
    try:
        df = pd.DataFrame(columns=['cafe_name', "cafe_type", 'starring', 'visitor_review', 'blog_review', 'address', 'business_hours', 'phone_number', 'connection_site', 'description',
                                    "menu", "keyword", 'keyword_number', 'image_link'])
        # WebDriver 초기화
        driver = webdriver.Chrome() # Chrome 드라이버 사용
        driver.get(f"https://map.naver.com/p/search/{cafe_name} 카페") # 네이버 지도 접속

        # 결과 로드 대기
        time.sleep(3) # 페이지 로드를 위해 잠시 대기


        while True:
            # searchIframe으로 컨텍스트 전환 + div scroll 내리기
            driver.switch_to.frame("searchIframe")
            scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
            for _ in range(5):
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll_div)
                time.sleep(1)

            # 목록을 담고 있는 ul 태그를 찾고, 그 안의 모든 li 태그를 가져옴
            cafe_list = scroll_div.find_elements(By.TAG_NAME, "li")
            cafe_size = len(cafe_list)


            for item in cafe_list:
                try:
                    place_bluelink = item.find_element(By.CLASS_NAME, "place_bluelink")
                    # 부모의 부모인 a 태그 찾기 및 클릭
                    parent_a_tag = place_bluelink.find_element(By.XPATH, "../..")
                    parent_a_tag.click()

                    time.sleep(3)

                    # 필요한 작업 수행 (예: 상세 정보 수집)
                    driver.switch_to.default_content()
                    driver.switch_to.frame("entryIframe")
                    cafe_row = cafe_crawling2()
                    if cafe_row:
                        df.loc[len(df)] = cafe_row
                        print(cafe_row)

                    # 다음 카페로 이동하기 전에 searchIframe으로 다시 컨텍스트 전
                    driver.switch_to.default_content()
                    driver.switch_to.frame("searchIframe")

                except Exception as e:
                    print(e)
                    pass

            next_a_link = driver.find_elements(By.CLASS_NAME, "eUTV2")[1]
            aria_disabled = next_a_link.get_attribute("aria-disabled")
            # 'aria-disabled'가 'false'이면 a 태그 클릭
            if aria_disabled == "false":
                next_a_link.click()
            else:
                break

            # 원래 프레임으로 돌아감
            driver.switch_to.default_content()
    finally:
            df.to_csv(f"./new_cafe_data/{cafe_name}.csv", encoding='utf-8')


['카페크림', '카페,디저트', 0, 181, 66, '대구 달서구 서당로3길 70 1층 카페크림', '10:30에 영업 시작', '0507-1308-4244', '', '', '', {'디저트가 맛있어요': 122, '친절해요': 116, '매장이 청결해요': 73, '특별한 메뉴가 있어요': 72, '커피가 맛있어요': 56, '인테리어가 멋져요': 52, '음료가 맛있어요': 49, '가성비가 좋아요': 37, '사진이 잘 나와요': 32, '대화하기 좋아요': 31, '좌석이 편해요': 29, '뷰가 좋아요': 20, '집중하기 좋아요': 12, '주차하기 편해요': 7, '화장실이 깨끗해요': 7, '선물하기 좋아요': 3, '아늑해요': 2, '주문제작을 잘해줘요': 1, '음악이 좋아요': 1, '비싼 만큼 가치있어요': 1, '매장이 넓어요': 1, '종류가 다양해요': 1}, 180, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230114_34%2F1673630133246P0ON5_JPEG%2F006056E0-310A-4DB0-B933-F0A652B74728.jpeg"']
['노다웃', '카페', 4.67, 474, 316, '대구 달서구 서당로7길 64 1층 No doubt', '10:00에 영업 시작', '070-4158-7777', '', '', '', {'커피가 맛있어요': 254, '친절해요': 129, '음료가 맛있어요': 81, '매장이 청결해요': 79, '디저트가 맛있어요': 59, '인테리어가 멋져요': 52, '가성비가 좋아요': 49, '특별한 메뉴가 있어요': 45, '대화하기 좋아요': 36, '사진이 잘 나와요': 16, '집중하기 좋아요': 16, '좌석이 편해요': 10, '뷰가 좋아요': 9, '아늑해요': 3, '차분한 분위기에요': 2, '주차하기

['투썸플레이스 대구계명대병원점', '카페', 4.6, 903, 21, '대구 달서구 달구벌대로 1019 2층', '09:00에 영업 시작', '053-721-6422', '', '', '', {'커피가 맛있어요': 250, '디저트가 맛있어요': 180, '음료가 맛있어요': 144, '매장이 청결해요': 114, '대화하기 좋아요': 112, '친절해요': 105, '좌석이 편해요': 102, '집중하기 좋아요': 84, '인테리어가 멋져요': 58, '뷰가 좋아요': 56, '주차하기 편해요': 32, '특별한 메뉴가 있어요': 21, '화장실이 깨끗해요': 20, '가성비가 좋아요': 17, '사진이 잘 나와요': 13, '매장이 넓어요': 1}, 673, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200404_286%2F1585990934011Q5wJu_JPEG%2FMx1a7TpykBaRdKWAnJl1y3Sr.jpg"']
['론데일2.0', '카페,디저트', 0, 74, 55, '대구 달서구 서당로7길 56 2층', '12:00에 영업 시작', '010-2204-4570', '', '', '', {'디저트가 맛있어요': 53, '커피가 맛있어요': 43, '인테리어가 멋져요': 30, '친절해요': 29, '음료가 맛있어요': 28, '사진이 잘 나와요': 21, '매장이 청결해요': 13, '뷰가 좋아요': 12, '대화하기 좋아요': 11, '특별한 메뉴가 있어요': 10, '좌석이 편해요': 8, '화장실이 깨끗해요': 7, '주차하기 편해요': 4, '집중하기 좋아요': 4, '빵이 맛있어요': 1, '아늑해요': 1, '음식이 맛있어요': 1, '포장이 깔끔해요': 1, '가성비가 좋아요': 1, '단체모임 하기 좋아요': 1}, 74, '"https://search.pstatic.

['스타벅스 계명대동산병원점', '카페', 4.55, 4103, 55, '대구 달서구 달구벌대로 1095', '07:00에 영업 시작', '1522-3232', '', '', '', {'커피가 맛있어요': 1262, '친절해요': 633, '음료가 맛있어요': 529, '매장이 청결해요': 391, '디저트가 맛있어요': 387, '대화하기 좋아요': 201, '좌석이 편해요': 195, '주차하기 편해요': 124, '인테리어가 멋져요': 123, '집중하기 좋아요': 114, '특별한 메뉴가 있어요': 93, '가성비가 좋아요': 73, '화장실이 깨끗해요': 72, '뷰가 좋아요': 41, '사진이 잘 나와요': 33, '음식이 맛있어요': 1, '비싼 만큼 가치있어요': 1, '매장이 넓어요': 1, '오래 머무르기 좋아요': 1}, 2474, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20190828_93%2F1566953601239OT9MQ_PNG%2FxX7Wv642gXMoTI0DAv0hRymS.png"']
['엔제리너스 대구계명대점', '카페', 4.55, 575, 25, '대구 달서구 계대동문로 2', '09:30에 영업 시작', '053-582-5078', '', '', '', {'커피가 맛있어요': 123, '집중하기 좋아요': 88, '친절해요': 82, '디저트가 맛있어요': 75, '음료가 맛있어요': 69, '매장이 청결해요': 59, '좌석이 편해요': 52, '대화하기 좋아요': 41, '가성비가 좋아요': 40, '특별한 메뉴가 있어요': 36, '뷰가 좋아요': 32, '화장실이 깨끗해요': 24, '인테리어가 멋져요': 16, '사진이 잘 나와요': 5, '주차하기 편해요': 2}, 287, '"https://search.pstatic.net/common/?autoRot

['Kaffi 707', '카페', 4.65, 113, 26, '대구 달서구 서당로7길 70', '11:00에 영업 시작', '053-218-1707', '', '', '', {'커피가 맛있어요': 35, '음료가 맛있어요': 28, '친절해요': 26, '디저트가 맛있어요': 16, '가성비가 좋아요': 15, '대화하기 좋아요': 13, '매장이 청결해요': 9, '인테리어가 멋져요': 5, '좌석이 편해요': 3, '집중하기 좋아요': 2, '아늑해요': 1}, 71, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200322_279%2F15848752554807hySg_JPEG%2FZfAEYuQjm0iKcbmkCu-EbAq-.jpg"']
['아메리칸트레일러 대구동산의료원점', '과일,주스전문점', 4.69, 169, 14, '대구 달서구 달구벌대로 1035 지하1층 푸드코트 입구 앞', '06:30에 영업 시작', '053-258-5841', '', '', '', {'음료가 맛있어요': 63, '친절해요': 35, '재료가 신선해요': 26, '특별한 메뉴가 있어요': 11, '매장이 청결해요': 11, '가성비가 좋아요': 10, '디저트가 맛있어요': 7, '주차하기 편해요': 3, '화장실이 깨끗해요': 3, '좌석이 편해요': 2, '음식이 맛있어요': 1, '인테리어가 멋져요': 1, '양이 많아요': 1, '사진이 잘 나와요': 1, '대화하기 좋아요': 1, '건강한 맛이에요': 1, '종류가 다양해요': 1}, 105, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200324_37%2F1585041149335SBPT1_PNG

['파리바게뜨 대구호산점', '베이커리', 4.45, 607, 13, '대구 달서구 호산동로 186 (호산동) .', '07:00에 영업 시작', '053-581-4714', '', '', '', {'빵이 맛있어요': 207, '친절해요': 107, '매장이 청결해요': 58, '가성비가 좋아요': 41, '커피가 맛있어요': 32, '음료가 맛있어요': 26, '특별한 메뉴가 있어요': 26, '인테리어가 멋져요': 22, '주차하기 편해요': 18, '대화하기 좋아요': 18, '좌석이 편해요': 16, '뷰가 좋아요': 11, '집중하기 좋아요': 10, '사진이 잘 나와요': 8, '화장실이 깨끗해요': 5}, 348, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220413_156%2F1649839382313Keayi_PNG%2Fimage.png"']
['마시그래이 성서계대정문점', '카페', 4.4, 312, 9, '대구 달서구 호산동로35북길 50 마시그래이 성서계대정문점', '09:30에 영업 시작', '0507-1331-2950', '', '', '', {'음료가 맛있어요': 102, '가성비가 좋아요': 100, '커피가 맛있어요': 96, '친절해요': 75, '매장이 청결해요': 38, '디저트가 맛있어요': 23, '대화하기 좋아요': 17, '인테리어가 멋져요': 11, '집중하기 좋아요': 11, '뷰가 좋아요': 11, '좌석이 편해요': 9, '사진이 잘 나와요': 7, '특별한 메뉴가 있어요': 7, '화장실이 깨끗해요': 5, '주차하기 편해요': 3}, 252, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F202

...

['파리바게뜨 대구신천점', '베이커리', 4.55, 1698, 3, '대구 동구 동부로 26 (신천동, 신천역센트럴리버파크) .', '', '053-252-3990', '', '', '', {'빵이 맛있어요': 704, '친절해요': 282, '매장이 청결해요': 147, '가성비가 좋아요': 88, '커피가 맛있어요': 59, '음료가 맛있어요': 55, '특별한 메뉴가 있어요': 48, '주차하기 편해요': 23, '인테리어가 멋져요': 21, '대화하기 좋아요': 16, '뷰가 좋아요': 10, '좌석이 편해요': 7, '사진이 잘 나와요': 6, '집중하기 좋아요': 6, '화장실이 깨끗해요': 6, '디저트가 맛있어요': 1}, 987, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220413_281%2F1649837532553d3KbN_PNG%2Fimage.png"']
['바겟팜 신세계 동대구복합환승센터점', '카페,디저트', 0, 84, 13, '대구 동구 동부로 149', '', '', '', '', '', {'디저트가 맛있어요': 39, '친절해요': 31, '특별한 메뉴가 있어요': 18, '가성비가 좋아요': 11, '커피가 맛있어요': 7, '주차하기 편해요': 7, '매장이 청결해요': 7, '음료가 맛있어요': 5, '빵이 맛있어요': 2, '좌석이 편해요': 1, '재료가 신선해요': 1, '인테리어가 멋져요': 1}, 81, '']
['빽다방 대구파티마삼거리점', '테이크아웃커피', 0, 44, 1, '대구 동구 동대구로 590', '07:00에 영업 시작', '0507-0288-7370', '', '', '', {'커피가 맛있어요': 26, '가성비가 좋아요': 15, '음료가 맛있어요': 14, '친절해요': 13, '디저트가 맛있어요': 6, '매장이 청결해요': 6

['StoryWay 동대구맞이다인경주', '호두과자', 0, 279, 0, '대구 동구 신암동 259-26', '', '', '', '', '', {'친절해요': 88, '디저트가 맛있어요': 67, '특별한 메뉴가 있어요': 38, '매장이 청결해요': 37, '가성비가 좋아요': 36, '커피가 맛있어요': 35, '음료가 맛있어요': 18, '좌석이 편해요': 7, '주차하기 편해요': 6, '대화하기 좋아요': 5, '화장실이 깨끗해요': 5, '인테리어가 멋져요': 4, '집중하기 좋아요': 3, '뷰가 좋아요': 2}, 263, '']
['스토리웨이 동대구신맞이빵장수꽈베기', '베이커리', 0, 705, 0, '대구 동구 신암동 259-26', '', '', '', '', '', {'빵이 맛있어요': 543, '친절해요': 169, '특별한 메뉴가 있어요': 106, '가성비가 좋아요': 84, '매장이 청결해요': 74, '커피가 맛있어요': 32, '음료가 맛있어요': 29, '주차하기 편해요': 12, '대화하기 좋아요': 7, '화장실이 깨끗해요': 6, '뷰가 좋아요': 6, '좌석이 편해요': 5, '인테리어가 멋져요': 3, '음식이 맛있어요': 2, '사진이 잘 나와요': 2, '집중하기 좋아요': 2, '디저트가 맛있어요': 1, '음식이 빨리 나와요': 1, '재료가 신선해요': 1, '양이 많아요': 1, '건강한 맛이에요': 1, '종류가 다양해요': 1}, 688, '']
['인잇 현대시티아울렛 대구점', '카페,디저트', 0, 135, 18, '대구 동구 동대구로 454 현대시티아울렛 대구점 2층', '10:30에 영업 시작', '053-430-2189', '', '', '', {'커피가 맛있어요': 57, '주차하기 편해요': 36, '매장이 청결해요': 30, '대화하기 좋아요': 25, '좌석이 편해요': 24, '친절해요': 24, '디저트가 맛있어요': 20, '음료가 맛있어요': 19, '인테리어가 멋져요': 14

['컴포즈커피 대구신천점', '카페', 0, 260, 4, '대구 동구 동부로22길 72', '08:00에 영업 시작', '0507-0288-7677', '', '', '', {'커피가 맛있어요': 138, '가성비가 좋아요': 94, '음료가 맛있어요': 76, '친절해요': 72, '매장이 청결해요': 44, '디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 14, '대화하기 좋아요': 13, '좌석이 편해요': 12, '인테리어가 멋져요': 9, '주차하기 편해요': 4, '사진이 잘 나와요': 4, '집중하기 좋아요': 4, '뷰가 좋아요': 2, '양이 많아요': 1, '음악이 좋아요': 1, '화장실이 깨끗해요': 1, '종류가 다양해요': 1}, 234, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220712_24%2F1657608043612zOjV2_JPEG%2FKakaoTalk_20220712_153705714.jpg"']
['시아', '카페,디저트', 0, 42, 13, '대구 동구 경대로 78-1', '11/17 휴무', '0507-1433-5124', '', '', '', {'커피가 맛있어요': 19, '친절해요': 14, '음료가 맛있어요': 10, '가성비가 좋아요': 8, '인테리어가 멋져요': 5, '매장이 청결해요': 3, '디저트가 맛있어요': 2, '집중하기 좋아요': 2, '좌석이 편해요': 1, '재료가 신선해요': 1, '포장이 깔끔해요': 1, '특별한 메뉴가 있어요': 1, '대화하기 좋아요': 1, '뷰가 좋아요': 1}, 35, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221107_36

['인잇 현대시티아울렛 대구점', '카페,디저트', 0, 135, 18, '대구 동구 동대구로 454 현대시티아울렛 대구점 2층', '10:30에 영업 시작', '053-430-2189', '', '', '', {'커피가 맛있어요': 57, '주차하기 편해요': 36, '매장이 청결해요': 30, '대화하기 좋아요': 25, '좌석이 편해요': 24, '친절해요': 24, '디저트가 맛있어요': 20, '음료가 맛있어요': 19, '인테리어가 멋져요': 14, '뷰가 좋아요': 10, '화장실이 깨끗해요': 8, '사진이 잘 나와요': 7, '가성비가 좋아요': 7, '특별한 메뉴가 있어요': 6, '집중하기 좋아요': 5, '야외 공간이 멋져요': 1}, 130, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220926_172%2F1664174482603GkHgk_JPEG%2FKakaoTalk_20220908_104030486.jpg"']
['Coffee IN', '카페,디저트', 4.61, 34, 0, '대구 동구 동대구로 550 동대구역', '', '', '', '', '', {'커피가 맛있어요': 11, '친절해요': 6, '음료가 맛있어요': 4, '매장이 청결해요': 2, '좌석이 편해요': 1, '인테리어가 멋져요': 1, '사진이 잘 나와요': 1, '가성비가 좋아요': 1, '특별한 메뉴가 있어요': 1, '집중하기 좋아요': 1, '화장실이 깨끗해요': 1}, 20, '']
['맥 동대구구맞이호두과자점', '호두과자', 4.23, 18, 0, '대구 동구 동대구로 550', '', '', '', '', '', '', '', '']
['코레일유통 동대구구맞이방도너츠', '도넛', 4.3, 480, 1, '대구 동구 신암동 259-26', '', '', '', '', '', {'커피가 맛

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x00007FF7B51582B2+55298]
	(No symbol) [0x00007FF7B50C5E02]
	(No symbol) [0x00007FF7B4F805AB]
	(No symbol) [0x00007FF7B4F60038]
	(No symbol) [0x00007FF7B4FE6BC7]
	(No symbol) [0x00007FF7B4FFA15F]
	(No symbol) [0x00007FF7B4FE1E83]
	(No symbol) [0x00007FF7B4FB670A]
	(No symbol) [0x00007FF7B4FB7964]
	GetHandleVerifier [0x00007FF7B54D0AAB+3694587]
	GetHandleVerifier [0x00007FF7B552728E+4048862]
	GetHandleVerifier [0x00007FF7B551F173+4015811]
	GetHandleVerifier [0x00007FF7B51F47D6+695590]
	(No symbol) [0x00007FF7B50D0CE8]
	(No symbol) [0x00007FF7B50CCF34]
	(No symbol) [0x00007FF7B50CD062]
	(No symbol) [0x00007FF7B50BD3A3]
	BaseThreadInitThunk [0x00007FFE372D257D+29]
	RtlUserThreadStart [0x00007FFE38B0AA78+40]
